### Gas demand correlations ###

Using NGrid data from Hazel (Thornton et al., 2019) for exploring the decadal relationship between different climate predictors (surface variables, indicies etc.) and UK gas demand (which has been postprocessed to remove non-linear trends).

In [1]:
# Local libraries
import os
import sys
import glob

# Third-party libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import pearsonr

These data are in .dat format, so we will need to convert this into .csv with NaNs using Pandas.

In [2]:
# Print the files in the directory
dir = "/home/users/benhutch/NGrid_demand"

# List the files in this directory
os.listdir(dir)

['Electricity_demand_5yrRmean_full_GB.dat',
 'Electricity_demand_5yrRmean_full_nonhol_GB.dat',
 'Gas_demand_5yrRmean_full_GB.dat',
 'Gas_demand_5yrRmean_full_nonhol_GB.dat',
 'Gas_raw_demand_GB.dat']

### Data description ###

#### Electricity demand ####

* *Electricity_demand_5yrRmean_full_GB.dat* - daily UK electricity demand data with the 5-year running mean removed including data for weekends and holidays.
* *Electricity_demand_5yrRmean_full_nonhol_GB.dat* - daily UK electricity demand data with the 5-year running mean removed *not* including data for weekends and holidays.

#### Gas demand ####

* *Gas_raw_demand_GB.dat* - daily gas demand for the UK.
* *Gas_demand_5yrRmean_full_GB.dat* - daily gas demand for the UK with the 5-year running mean removed.
* *Gas_demand_5yrRmean_full_nonhol_GB.dat* - daily gas demand for the UK with the 5-year running mean removed *not* including data for weekends and holidays.

In [3]:
# We want to load these all into a .csv file
raw_gas_file = "Gas_raw_demand_GB.dat"

# load using pandas
df_raw_gas = pd.read_csv(os.path.join(dir, raw_gas_file), sep=" ", header=0)

In [4]:
df_raw_gas.head()

,date,demand_MCM
1,1998-01-01,279.43039
2,1998-01-02,270.04742
3,1998-01-03,279.82163
4,1998-01-04,308.54917
5,1998-01-05,341.50085


In [5]:
# Try loading the data with NaN values
gas_file_nohols = "Gas_demand_5yrRmean_full_nonhol_GB.dat"

# Load in the .csv file
df_gas_nohols = pd.read_csv(os.path.join(dir, gas_file_nohols), sep=" ", header=0, na_values="NA")

In [6]:
df_gas_nohols.head()

,date,demand_full_5yrRmean
1,1998-01-01,NaN
2,1998-01-02,NaN
3,1998-01-03,NaN
4,1998-01-04,NaN
5,1998-01-05,NaN


In [7]:
# laod the other data
gas_demand_with_hols = "Gas_demand_5yrRmean_full_GB.dat"

# Load in the .csv file
df_gas_hols = pd.read_csv(os.path.join(dir, gas_demand_with_hols), sep=" ", header=0, na_values="NA")


In [8]:
df_gas_hols.head()

,date,demand_full_5yrRmean
1,1998-01-01,NaN
2,1998-01-02,NaN
3,1998-01-03,NaN
4,1998-01-04,NaN
5,1998-01-05,NaN


In [9]:
# Convert the date to a datetime object
df_gas_hols["date"] = pd.to_datetime(df_gas_hols["date"], format="%Y-%m-%d")


In [10]:
df_gas_hols

,date,demand_full_5yrRmean
1,1998-01-01,NaN
2,1998-01-02,NaN
3,1998-01-03,NaN
4,1998-01-04,NaN
5,1998-01-05,NaN
...,...,...
5475,2012-12-27,NaN
5476,2012-12-28,NaN
5477,2012-12-29,NaN
5478,2012-12-30,NaN


In [11]:
# Convert the date to a datetime object
df_gas_nohols["date"] = pd.to_datetime(df_gas_nohols["date"], format="%Y-%m-%d")

In [12]:
# Same for the raw data
df_raw_gas["date"] = pd.to_datetime(df_raw_gas["date"], format="%Y-%m-%d")

In [13]:
# For nohols add the suffix _nohols to the demand_full_5yrRmean column
df_gas_nohols.rename(columns={"demand_full_5yrRmean": "demand_full_5yrRmean_nohols"}, inplace=True)

In [14]:
# join the three dataframes on the date column
df_gas = pd.merge(df_gas_hols, df_gas_nohols, on="date")

# merge this with the raw data
df_gas = pd.merge(df_gas, df_raw_gas, on="date")

In [15]:
df_gas.head()

,date,demand_full_5yrRmean,demand_full_5yrRmean_nohols,demand_MCM
0,1998-01-01,NaN,NaN,279.43039
1,1998-01-02,NaN,NaN,270.04742
2,1998-01-03,NaN,NaN,279.82163
3,1998-01-04,NaN,NaN,308.54917
4,1998-01-05,NaN,NaN,341.50085


In [16]:
# rename demand_MCM as gas_demand_raw
df_gas.rename(columns={"demand_MCM": "gas_demand_raw"}, inplace=True)

# rename demand_full_5yrRmean as gas_demand_5yrRmean
df_gas.rename(columns={"demand_full_5yrRmean": "gas_demand_5yrRmean"}, inplace=True)

# rename demand_full_5yrRmean_nohols as gas_demand_5yrRmean_nohols
df_gas.rename(columns={"demand_full_5yrRmean_nohols": "gas_demand_5yrRmean_nohols"}, inplace=True)

In [17]:
df_gas.head()

,date,gas_demand_5yrRmean,gas_demand_5yrRmean_nohols,gas_demand_raw
0,1998-01-01,NaN,NaN,279.43039
1,1998-01-02,NaN,NaN,270.04742
2,1998-01-03,NaN,NaN,279.82163
3,1998-01-04,NaN,NaN,308.54917
4,1998-01-05,NaN,NaN,341.50085


In [18]:
# Load in the electricity demand data
elec_file = "Electricity_demand_5yrRmean_full_nonhol_GB.dat"

# Load in the .csv file
df_elec_nohols = pd.read_csv(os.path.join(dir, elec_file), sep=" ", header=0, na_values="NA")

In [19]:
df_elec_nohols.head()

,date,demand_full_5yrRmean
1,1975-01-01,NaN
2,1975-01-02,NaN
3,1975-01-03,NaN
4,1975-01-04,NaN
5,1975-01-05,NaN


In [20]:
# rename demand_full_5yrRmean as elec_demand_5yrRmean_nohols
df_elec_nohols.rename(columns={"demand_full_5yrRmean": "elec_demand_5yrRmean_nohols"}, inplace=True)

In [21]:
# laod the other data
elec_demand_with_hols = "Electricity_demand_5yrRmean_full_GB.dat"

# Load in the .csv file
df_elec_hols = pd.read_csv(os.path.join(dir, elec_demand_with_hols), sep=" ", header=0, na_values="NA")

In [22]:
df_elec_hols.head()

,date,demand_full_5yrRmean
1,1975-01-01,NaN
2,1975-01-02,NaN
3,1975-01-03,NaN
4,1975-01-04,NaN
5,1975-01-05,NaN


In [23]:
# Convert the date to datetime
df_elec_hols["date"] = pd.to_datetime(df_elec_hols["date"], format="%Y-%m-%d")

In [24]:
# Convert the date to datetime
df_elec_nohols["date"] = pd.to_datetime(df_elec_nohols["date"], format="%Y-%m-%d")

In [25]:
# Merge the two dataframes on the date column
df_elec = pd.merge(df_elec_hols, df_elec_nohols, on="date")

In [26]:
df_elec

,date,demand_full_5yrRmean,elec_demand_5yrRmean_nohols
0,1975-01-01,NaN,NaN
1,1975-01-02,NaN,NaN
2,1975-01-03,NaN,NaN
3,1975-01-04,NaN,NaN
4,1975-01-05,NaN,NaN
...,...,...,...
14056,2013-06-26,NaN,NaN
14057,2013-06-27,NaN,NaN
14058,2013-06-28,NaN,NaN
14059,2013-06-29,NaN,NaN


In [27]:
# rename demand_full_5yrRmean as elec_demand_5yrRmean
df_elec.rename(columns={"demand_full_5yrRmean": "elec_demand_5yrRmean"}, inplace=True)

In [28]:
# Merge the two dataframes on 'date' column
df_merged = df_elec.merge(df_gas, on='date', how='left')

In [29]:
df_merged

,date,elec_demand_5yrRmean,elec_demand_5yrRmean_nohols,gas_demand_5yrRmean,gas_demand_5yrRmean_nohols,gas_demand_raw
0,1975-01-01,NaN,NaN,NaN,NaN,NaN
1,1975-01-02,NaN,NaN,NaN,NaN,NaN
2,1975-01-03,NaN,NaN,NaN,NaN,NaN
3,1975-01-04,NaN,NaN,NaN,NaN,NaN
4,1975-01-05,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
14059,2013-06-26,NaN,NaN,NaN,NaN,NaN
14060,2013-06-27,NaN,NaN,NaN,NaN,NaN
14061,2013-06-28,NaN,NaN,NaN,NaN,NaN
14062,2013-06-29,NaN,NaN,NaN,NaN,NaN


In [30]:
# set the filename
output_file = "gas_electricity_demand_data.csv"

# output directory
output_dir = "/home/users/benhutch/NGrid_demand/csv_files"

# if the directory does not exist, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# save the dataframe to a .csv file
df_merged.to_csv(os.path.join(output_dir, output_file), index=False)